# 🔐 Secrets Manager


In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from datetime import datetime

# Simple API keys configuration
API_KEYS = {
    'NVIDIA_API_KEY': {'name': 'NVIDIA API Key', 'required': True, 'icon': '🎯'},
    'TAVILY_API_KEY': {'name': 'Tavily API Key', 'required': False, 'icon': '🔍'},
    'LANGSMITH_API_KEY': {'name': 'LangSmith API Key', 'required': False, 'icon': '📊'}
}

SECRETS_FILE = "../secrets.env"

# Create API key input widgets
api_key_widgets = {}

for key, config in API_KEYS.items():
    required_text = " (Required)" if config['required'] else " (Optional)"
    api_key_widgets[key] = widgets.Password(
        placeholder=f'Enter your {config["name"]}',
        description=f'{config["icon"]} {config["name"]}{required_text}:',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='500px', margin='10px')
    )

# Buttons
save_button = widgets.Button(
    description='💾 Save API Keys',
    button_style='success',
    layout=widgets.Layout(width='150px', height='40px', margin='10px'),
    disabled=True  # Start disabled
)

clear_button = widgets.Button(
    description='🗑️ Clear All', 
    button_style='warning',
    layout=widgets.Layout(width='150px', height='40px', margin='10px')
)

# Status output
status_output = widgets.Output(
    layout=widgets.Layout(width='100%', height='150px', border='1px solid #ccc', margin='10px')
)


In [3]:
# Track original values to detect changes
original_values = {}

def check_for_changes():
    """Enable save button if any values have changed"""
    has_changes = False
    for key, widget in api_key_widgets.items():
        if widget.value != original_values.get(key, ''):
            has_changes = True
            break
    
    save_button.disabled = not has_changes
    if has_changes:
        save_button.button_style = 'success'
        save_button.description = '💾 Save Changes'
    else:
        save_button.button_style = 'info'
        save_button.description = '💾 No Changes'

def on_value_change(change):
    """Called when any API key input changes"""
    check_for_changes()

# Simple functions for API key management
def save_api_keys(b):
    """Save API keys"""
    with status_output:
        clear_output()
        print(f"🕐 {datetime.now().strftime('%H:%M:%S')} - Saving API keys...")
        
        saved_keys = []
        env_content = []
        
        for key, widget in api_key_widgets.items():
            if widget.value.strip():
                os.environ[key] = widget.value.strip()
                env_content.append(f'{key}={widget.value.strip()}')
                config = API_KEYS[key]
                print(f"   ✓ {config['icon']} {config['name']} - Saved")
                saved_keys.append(config['name'])
        
        # Write to secrets.env file
        if env_content:
            with open(SECRETS_FILE, 'w') as f:
                f.write('\n'.join(env_content) + '\n')
            print(f"\n📁 Saved to {SECRETS_FILE}")
        
        # Update original values after save
        for key, widget in api_key_widgets.items():
            original_values[key] = widget.value
        
        if saved_keys:
            print(f"\n🎉 SUCCESS! {len(saved_keys)} API keys saved.")
            print(f"🚀 Ready to use in your notebooks!")
        else:
            print("⚠️  No API keys provided.")
        
        # Disable save button after successful save
        check_for_changes()

# Connect button events
save_button.on_click(save_api_keys)

# Add change listeners to all API key widgets
for widget in api_key_widgets.values():
    widget.observe(on_value_change, names='value')



In [4]:
form_widgets = []

# Display the dashboard
for widget in api_key_widgets.values():
    form_widgets.append(widget)

# Buttons
form_widgets.append(widgets.HBox([save_button]))

form = widgets.VBox(form_widgets)

tabs = widgets.Tab()
tabs.children = [form, status_output]
tabs.titles = ['Secrets Manager', 'Logs']

display(tabs)

# Load existing values if available
for key in api_key_widgets.keys():
    existing_value = os.getenv(key, '')
    api_key_widgets[key].value = existing_value
    original_values[key] = existing_value

# Initial button state check
check_for_changes()